# init

In [1]:
import numpy as np
import pandas as pd
import polars as pl
from timeit import timeit
import os, sys
print(os.getcwd())
os.chdir('/gpfs/data/healthcare-allocate/CLIF-MIMIC/code')
print(os.getcwd())

proj_root = "/gpfs/data/healthcare-allocate/CLIF-MIMIC"
if proj_root not in sys.path:
    sys.path.append(proj_root)

from code.custom_utils import *

/gpfs/data/healthcare-allocate/CLIF-MIMIC
/gpfs/data/healthcare-allocate/CLIF-MIMIC/code


In [2]:
d_items = pd.read_csv("../mimic-iv-2.2/icu/d_items.csv.gz")
chartevents = pd.read_parquet('../mimic-iv-2.2/icu/chartevents.parquet')

def load_mimic_table(module, table):
    return pd.read_parquet(f'../mimic-iv-2.2/{module}/{table}.parquet')

# utils

## `ItemFinder`

In [12]:
class ItemFinder():
    def __init__(self, kw, df = d_items, col: str = "label", 
                 case: bool = False):
        '''
        Look up an item by keyword from the `d_items` table of the `icu` module.
        - df = `d_items`
        - col = {"label", "abbr"}
        '''
        self.kw = kw
        self.df = df
        self.col = "abbreviation" if col == "abbr" else col
        self.case = case

        self.kw_items = df[
            df[self.col].str.contains(kw, case = case, na = False)
        ]
        self.kw_items_ids = self.kw_items["itemid"].values
        self.kw_items_chartevents = chartevents.loc[
            chartevents["itemid"].isin(self.kw_items_ids),
            :
        ]
        self.item_freq = self.kw_items_chartevents.value_counts("itemid")

        self.candidate_table = (
            self.kw_items
            .loc[:, ["itemid", "label", "abbreviation", "linksto", "category", "unitname", "param_type"]]
            .join(self.item_freq, on = "itemid", validate = "1:1")
            .sort_values(by = "count", ascending = False) 
            .assign(
                value_instances = lambda x: x["itemid"].apply(ItemFinder.item_id_to_value_instances)
            )
        )

    @staticmethod
    def item_id_to_label(item_id: int) -> str:
        '''
        Return the "label" string given an item_id string 
        '''
        return d_items.loc[d_items["itemid"] == item_id, "label"].values[0]

    @staticmethod
    def item_id_to_chartevents(item_id: int):
        '''
        Return all the chartevents associated with this item id in a pandas df.
        '''
        return chartevents.loc[chartevents["itemid"] == item_id, :]

    @staticmethod
    def item_id_to_value_instances(item_id: int):
        '''
        Wrapper
        '''
        label = ItemFinder.item_id_to_label(item_id)

        param_type = d_items.loc[d_items["itemid"] == item_id, "param_type"].values[0]
        if param_type == "Numeric":
            val_instances = ItemFinder.item_id_to_value_instances_numeric(item_id)
        elif param_type == "Text":
            val_instances = ItemFinder.item_id_to_value_instances_categorical(item_id)
        else:
            return param_type
        print(f"item label: {label}; value instances: {str(val_instances)}")
        return val_instances

    @staticmethod
    def item_id_to_value_instances_categorical(item_id: int):
        '''
        Return all the unique categories
        '''
        assoc_chartevents = chartevents.loc[chartevents["itemid"] == item_id, :]
        categories = assoc_chartevents.value_counts("value").index.to_list()
        return str(categories)
    
    @staticmethod
    def item_id_to_value_instances_numeric(item_id: int):
        '''
        Find max, min, mean of a continuous, or numeric, item.
        '''
        valuenum_col = chartevents.loc[chartevents["itemid"] == item_id, :]["valuenum"]
        val_max, val_min, val_mean = valuenum_col.max(), valuenum_col.min(), valuenum_col.mean()
        return f"Max: {val_max}, Min: {val_min}, Mean: {val_mean}"

In [9]:
ItemFinder("Temperature").candidate_table

item label is Temperature Site
categories are ['Oral', 'Blood', 'Axillary', 'Rectal', 'Esophogeal', 'Temporal', 'Tympanic']
item label is Temperature Fahrenheit
item label is Skin Temperature
categories are ['Warm', 'Cool', 'Hot', 'Cold']
item label is Temperature Celsius
item label is Blood Temperature CCO (C)
item label is Cerebral Temperature (C)
item label is Changes in Temperature
categories are ['No report of temperature change', 'Reports feeling cold, hands cold and clammy to touch', 'Uncontrolled shivering']
item label is TemperatureF_ApacheIV
item label is Pt. Temperature (BG) (SOFT)


,itemid,label,abbreviation,category,unitname,param_type,count,value_instances
767,224642,Temperature Site,Temp Site,Routine Vital Signs,NaN,Text,1842387.0,"['Oral', 'Blood', 'Axillary', 'Rectal', 'Esoph..."
337,223761,Temperature Fahrenheit,Temperature F,Routine Vital Signs,°F,Numeric,1515962.0,"Max: 234123.0, Min: -99.9, Mean: 98.7157890197..."
505,224027,Skin Temperature,Skin Temp,Skin - Assessment,NaN,Text,952619.0,"['Warm', 'Cool', 'Hot', 'Cold']"
338,223762,Temperature Celsius,Temperature C,Routine Vital Signs,°C,Numeric,264628.0,"Max: 987.4, Min: 0.0, Mean: 37.05203024169777"
1814,226329,Blood Temperature CCO (C),Blood Temp CCO (C),Routine Vital Signs,°C,Numeric,151805.0,"Max: 336.7, Min: 0.0, Mean: 37.07423083165904"
3466,229236,Cerebral Temperature (C),Cerebral T (C),Hemodynamics,°C,Numeric,2469.0,"Max: 39.47, Min: 32.9, Mean: 37.076647225597405"
790,224674,Changes in Temperature,Changes in Temperature,Toxicology,NaN,Text,1972.0,"['No report of temperature change', 'Reports f..."
2097,227054,TemperatureF_ApacheIV,TemperatureF_ApacheIV,Scores - APACHE IV (2),°F,Numeric,7.0,"Max: 103.6, Min: 96.0, Mean: 99.8"
2776,228242,Pt. Temperature (BG) (SOFT),Pt. Temperature (BG) (SOFT),Labs,NaN,Numeric,NaN,"Max: nan, Min: nan, Mean: nan"


# by MIMIC modules / tables

## `hosp`

In [2]:
d_labitems = pd.read_csv("mimic-iv-2.2/hosp/d_labitems.csv.gz")

In [4]:
# d_labitems.head(10)
contains = d_labitems["label"].str.contains("temp", case=False, na=False)
d_labitems[contains]

,itemid,label,fluid,category
23,50825,Temperature,Blood,Blood Gas


In [19]:
d_labitems["fluid"].unique()
d_labitems["category"].unique()

array(['Blood Gas', 'Chemistry', 'Hematology'], dtype=object)

In [12]:
omr = pd.read_csv("../mimic-iv-2.2/hosp/omr.csv.gz")
omr.head(10)

,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-04-27,1,Blood Pressure,110/65
1,10000032,2180-04-27,1,Weight (Lbs),94
2,10000032,2180-05-07,1,BMI (kg/m2),18.0
3,10000032,2180-05-07,1,Height (Inches),60
4,10000032,2180-05-07,1,Weight (Lbs),92.15
5,10000032,2180-05-07,2,Weight (Lbs),92.15
6,10000032,2180-05-07,3,Weight (Lbs),92.15
7,10000032,2180-05-07,4,Weight (Lbs),92.15
8,10000032,2180-05-07,5,Weight (Lbs),92.15
9,10000032,2180-05-07,6,Weight (Lbs),92.15


## `icu`

### `d_items`

### `chartevents`

In [3]:
# resave into parquet
chartevents = load_gz_csv("mimic-iv-2.2/icu/chartevents.csv.gz")
chartevents.to_parquet('mimic-iv-2.2/icu/chartevents.parquet')  

In [54]:
chartevents.loc[
    chartevents["itemid"] == 224689, :]["valuenum"].agg(["max","min"])

max    2843.0
min       0.0
Name: valuenum, dtype: float64

In [ ]:
def map_one_item(item_id):
    '''
    create the rows in a vitals table corresponding to one item from MIMIC, based on item id.
    '''


In [47]:
item_id = 220050
vital_name = find_item_label(item_id)
vital_category = "sbp"
meas_site_name = itemid_to_meas_site_name[item_id]

vitals_sbp = (
    chartevents
    .loc[
        chartevents["itemid"] == item_id,  # :
        ["hadm_id", "charttime", "itemid", "value"]
    ]
    .rename(columns = {
        "hadm_id": "encounter_id", 
        "charttime": "recorded_dttm",
        "value": "vital_value"
        })
    .assign(
        vital_name = vital_name,
        vital_category = vital_category,
        meas_site_name = meas_site_name
    )
    .reindex(
        columns = ["encounter_id", "recorded_dttm", "vital_name", "vital_category", "vital_value", "meas_site_name"]
    )
)


In [48]:
vitals_sbp

,encounter_id,recorded_dttm,vital_name,vital_category,vital_value,meas_site_name
19182,23581541,2160-05-18 20:00:00,Arterial Blood Pressure systolic,sbp,116,arterial
19327,23581541,2160-05-18 21:00:00,Arterial Blood Pressure systolic,sbp,97,arterial
19353,23581541,2160-05-18 22:00:00,Arterial Blood Pressure systolic,sbp,123,arterial
19376,23581541,2160-05-18 22:10:00,Arterial Blood Pressure systolic,sbp,94,arterial
19385,23581541,2160-05-18 23:00:00,Arterial Blood Pressure systolic,sbp,116,arterial
...,...,...,...,...,...,...
313630748,26785317,2148-11-20 06:00:00,Arterial Blood Pressure systolic,sbp,124,arterial
313630778,26785317,2148-11-20 20:00:00,Arterial Blood Pressure systolic,sbp,104,arterial
313630874,26785317,2148-11-20 21:00:00,Arterial Blood Pressure systolic,sbp,102,arterial
313630901,26785317,2148-11-20 22:00:00,Arterial Blood Pressure systolic,sbp,105,arterial


### `d_items` counting

In [7]:
chartevents.loc[chartevents["itemid"] == 224642, "value"].unique()

array(['Oral', 'Axillary', 'Esophogeal', 'Tympanic', 'Rectal', 'Blood',
       'Temporal'], dtype=object)

In [15]:
chartevents.loc[
    (chartevents["itemid"].isin([224642,223762])) & (chartevents["stay_id"] == 39553978),
    :
]

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
18,10000032,29079034,39553978,66056.0,2180-07-23 20:00:00,2180-07-23 19:59:00,224642,Oral,NaN,None,0.0
160,10000032,29079034,39553978,88981.0,2180-07-23 14:00:00,2180-07-23 14:18:00,224642,Oral,NaN,None,0.0
370,10000032,29079034,39553978,88981.0,2180-07-23 17:00:00,2180-07-23 17:03:00,224642,Oral,NaN,None,0.0


In [24]:
temp = chartevents.loc[
    (chartevents["itemid"].isin([224642,223762])),
    :
]

temp

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
18,10000032,29079034,39553978,66056.0,2180-07-23 20:00:00,2180-07-23 19:59:00,224642,Oral,NaN,None,0.0
160,10000032,29079034,39553978,88981.0,2180-07-23 14:00:00,2180-07-23 14:18:00,224642,Oral,NaN,None,0.0
370,10000032,29079034,39553978,88981.0,2180-07-23 17:00:00,2180-07-23 17:03:00,224642,Oral,NaN,None,0.0
551,10000980,26913865,39765666,36518.0,2189-06-27 09:07:00,2189-06-27 09:07:00,224642,Axillary,NaN,None,0.0
733,10000980,26913865,39765666,36518.0,2189-06-27 12:00:00,2189-06-27 12:53:00,224642,Oral,NaN,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...
313644553,19999987,23865745,36195440,86756.0,2145-11-03 04:00:00,2145-11-03 04:24:00,224642,Oral,NaN,None,0.0
313644735,19999987,23865745,36195440,90295.0,2145-11-02 23:24:00,2145-11-02 23:24:00,224642,Oral,NaN,None,0.0
313644811,19999987,23865745,36195440,90295.0,2145-11-02 23:41:00,2145-11-02 23:41:00,224642,Oral,NaN,None,0.0
313644876,19999987,23865745,36195440,91879.0,2145-11-04 20:00:00,2145-11-04 19:38:00,224642,Oral,NaN,None,0.0


In [25]:
temp.loc[
    chartevents["itemid"] == 223762,
    :
]

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
8521,10001884,26184834,37510196,36518.0,2131-01-12 07:00:00,2131-01-12 08:07:00,223762,36.7,36.7,°C,0.0
8532,10001884,26184834,37510196,36518.0,2131-01-12 08:00:00,2131-01-12 08:07:00,223762,36.7,36.7,°C,0.0
8740,10001884,26184834,37510196,36518.0,2131-01-12 09:00:00,2131-01-12 09:06:00,223762,37,37.0,°C,0.0
8747,10001884,26184834,37510196,36518.0,2131-01-12 10:00:00,2131-01-12 10:21:00,223762,36.9,36.9,°C,0.0
8793,10001884,26184834,37510196,36518.0,2131-01-12 11:00:00,2131-01-12 11:26:00,223762,36.9,36.9,°C,0.0
...,...,...,...,...,...,...,...,...,...,...,...
313599244,19998843,24842066,30988867,64147.0,2187-02-07 14:00:00,2187-02-07 14:36:00,223762,38.7,38.7,°C,0.0
313599278,19998843,24842066,30988867,64147.0,2187-02-07 15:00:00,2187-02-07 15:12:00,223762,38.6,38.6,°C,0.0
313599292,19998843,24842066,30988867,64147.0,2187-02-07 16:00:00,2187-02-07 16:18:00,223762,38.5,38.5,°C,0.0
313599369,19998843,24842066,30988867,64147.0,2187-02-07 17:00:00,2187-02-07 17:10:00,223762,38.5,38.5,°C,0.0


# by CLIF tables

## `vitals`

### Routine Vital Signs

In [12]:
routine_vitals = ItemFinder("Routine Vital Signs", col = "category").candidate_table
routine_vitals

item label is Heart Rate
item label is Heart Rhythm
categories are ['SR (Sinus Rhythm)', 'ST (Sinus Tachycardia) ', 'AF (Atrial Fibrillation)', 'SB (Sinus Bradycardia)', 'V Paced', '1st AV (First degree AV Block) ', 'A Paced', 'AV Paced', 'A Flut (Atrial Flutter) ', 'RBBB (Right Bundle Branch Block) ', 'SA (Sinus Arrhythmia)', 'LBBB (Left Bundle Branch Block) ', 'JR (Junctional Rhythm)', '3rd AV (Complete Heart Block) ', 'SVT (Supra Ventricular Tachycardia)', '2nd AV W-M1 (Second degree AV Block Wenckebach - Mobitz1) ', 'JT (Junctional Tachycardia) ', 'WAP (Wandering atrial pacemaker)', '2nd AV M2 (Second degree AV Block - Mobitz 2) ', 'MAT (Multifocal atrial tachycardia)', 'VT (Ventricular Tachycardia) ', 'PAT (Paroxysmal Atrial Tachycardia)', 'Asystole', 'Idioventricular', 'VF (Ventricular Fibrillation) ', 'PJT (Paroxysmal Junctional Tachycardia) ']
item label is Ectopy Type 1
categories are ["PVC's", "PAC's", 'Vent. Bigeminy', 'Atrial Bigeminy', 'Vent. Trigeminy', 'Nod/Junc Escape',

,itemid,label,abbreviation,category,unitname,param_type,count,value_instances
2,220045,Heart Rate,HR,Routine Vital Signs,bpm,Numeric,6460839,"Max: 10000000.0, Min: -241395.0, Mean: 87.8015..."
5,220048,Heart Rhythm,Heart Rhythm,Routine Vital Signs,NaN,Text,5907987,"['SR (Sinus Rhythm)', 'ST (Sinus Tachycardia) ..."
772,224650,Ectopy Type 1,Ectopy Type 1,Routine Vital Signs,NaN,Text,5272404,"[""PVC's"", ""PAC's"", 'Vent. Bigeminy', 'Atrial B..."
24,220179,Non Invasive Blood Pressure systolic,NBPs,Routine Vital Signs,mmHg,Numeric,4067219,"Max: 1025100.0, Min: -69.0, Mean: 120.67245107..."
25,220180,Non Invasive Blood Pressure diastolic,NBPd,Routine Vital Signs,mmHg,Numeric,4066337,"Max: 114109.0, Min: -2.0, Mean: 66.418738289522"
26,220181,Non Invasive Blood Pressure mean,NBPm,Routine Vital Signs,mmHg,Numeric,4064537,"Max: 8888890.0, Min: -22767.0, Mean: 84.590976..."
8,220052,Arterial Blood Pressure mean,ABPm,Routine Vital Signs,mmHg,Numeric,2272013,"Max: 930000.0, Min: -135.0, Mean: 80.147497615..."
6,220050,Arterial Blood Pressure systolic,ABPs,Routine Vital Signs,mmHg,Numeric,2264001,"Max: 95119.0, Min: -94.0, Mean: 119.3417890414..."
7,220051,Arterial Blood Pressure diastolic,ABPd,Routine Vital Signs,mmHg,Numeric,2263647,"Max: 114100.0, Min: -40.0, Mean: 60.0630160091..."
767,224642,Temperature Site,Temp Site,Routine Vital Signs,NaN,Text,1842387,"['Oral', 'Blood', 'Axillary', 'Rectal', 'Esoph..."


### `respiratory_rate`

In [62]:
ItemFinder("respiratory rate").selected_items

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
28,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,NaN,NaN
799,224688,Respiratory Rate (Set),Respiratory Rate (Set),chartevents,Respiratory,insp/min,Numeric,NaN,NaN
800,224689,Respiratory Rate (spontaneous),Respiratory Rate (spontaneous),chartevents,Respiratory,insp/min,Numeric,NaN,NaN
801,224690,Respiratory Rate (Total),Respiratory Rate (Total),chartevents,Respiratory,insp/min,Numeric,NaN,36.0


In [64]:
find_summary_stats(224688)

Max: 1618.0, Min: -25.0, Mean: 18.738991028049398


### `temp_c`

In [67]:
find_value_instances(224027)

item label is Skin Temperature


Index(['Warm', 'Cool', 'Hot', 'Cold'], dtype='object', name='value')

In [ ]:
chartevents.loc[chartevents["charttime"] == "2180-07-23 20:00:00", :]

In [72]:
chartevents.loc[chartevents["caregiver_id"] == 66056.0, :]

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
9,10000032,29079034,39553978,66056.0,2180-07-23 19:00:00,2180-07-23 19:59:00,220045,97,97.0,bpm,0.0
10,10000032,29079034,39553978,66056.0,2180-07-23 19:00:00,2180-07-23 19:59:00,220179,93,93.0,mmHg,0.0
11,10000032,29079034,39553978,66056.0,2180-07-23 19:00:00,2180-07-23 19:59:00,220180,41,41.0,mmHg,0.0
12,10000032,29079034,39553978,66056.0,2180-07-23 19:00:00,2180-07-23 19:59:00,220181,56,56.0,mmHg,0.0
13,10000032,29079034,39553978,66056.0,2180-07-23 19:00:00,2180-07-23 19:59:00,220210,16,16.0,insp/min,0.0
...,...,...,...,...,...,...,...,...,...,...,...
313322602,19991135,29872770,33242276,66056.0,2133-07-08 07:00:00,2133-07-08 07:06:00,224650,PVC's,NaN,None,0.0
313322603,19991135,29872770,33242276,66056.0,2133-07-08 07:00:00,2133-07-08 07:06:00,224651,Occasional,NaN,None,0.0
313322604,19991135,29872770,33242276,66056.0,2133-07-08 07:01:00,2133-07-08 07:06:00,220179,166,166.0,mmHg,0.0
313322605,19991135,29872770,33242276,66056.0,2133-07-08 07:01:00,2133-07-08 07:06:00,220180,104,104.0,mmHg,0.0


In [70]:
find_assoc_chartevets(224642)

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
18,10000032,29079034,39553978,66056.0,2180-07-23 20:00:00,2180-07-23 19:59:00,224642,Oral,NaN,None,0.0
160,10000032,29079034,39553978,88981.0,2180-07-23 14:00:00,2180-07-23 14:18:00,224642,Oral,NaN,None,0.0
370,10000032,29079034,39553978,88981.0,2180-07-23 17:00:00,2180-07-23 17:03:00,224642,Oral,NaN,None,0.0
551,10000980,26913865,39765666,36518.0,2189-06-27 09:07:00,2189-06-27 09:07:00,224642,Axillary,NaN,None,0.0
733,10000980,26913865,39765666,36518.0,2189-06-27 12:00:00,2189-06-27 12:53:00,224642,Oral,NaN,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...
313644553,19999987,23865745,36195440,86756.0,2145-11-03 04:00:00,2145-11-03 04:24:00,224642,Oral,NaN,None,0.0
313644735,19999987,23865745,36195440,90295.0,2145-11-02 23:24:00,2145-11-02 23:24:00,224642,Oral,NaN,None,0.0
313644811,19999987,23865745,36195440,90295.0,2145-11-02 23:41:00,2145-11-02 23:41:00,224642,Oral,NaN,None,0.0
313644876,19999987,23865745,36195440,91879.0,2145-11-04 20:00:00,2145-11-04 19:38:00,224642,Oral,NaN,None,0.0


### `map`

In [10]:
ItemFinder("map", col="abbr").candidate_table

item label is IABP Mean
item label is MapApacheIIScore
item label is PotassiumApacheIIScore
item label is SodiumApacheIIScore
item label is MapApacheIIValue
item label is PotassiumApacheIIValue
item label is SodiumApacheIIValue
item label is MapScore_ApacheIV
item label is MAP_ApacheIV


,itemid,label,abbreviation,category,unitname,param_type,count,value_instances
635,224322,IABP Mean,MAP - Assisted,IABP,mmHg,Numeric,31605,"Max: 701.0, Min: 0.0, Mean: 80.09030216737858"
1975,226765,MapApacheIIScore,MapApacheIIScore,Scores - APACHE II,NaN,Numeric,19,"Max: 4.0, Min: 0.0, Mean: 1.7894736842105263"
1981,226771,PotassiumApacheIIScore,PotassiumApacheIIScore,Scores - APACHE II,NaN,Numeric,19,"Max: 1.0, Min: 0.0, Mean: 0.2631578947368421"
1985,226775,SodiumApacheIIScore,SodiumApacheIIScore,Scores - APACHE II,NaN,Numeric,19,"Max: 1.0, Min: 0.0, Mean: 0.10526315789473684"
1976,226766,MapApacheIIValue,MapApacheIIValue,Scores - APACHE II,mmHg,Numeric,18,"Max: 124.0, Min: 25.0, Mean: 62.611111111111114"
1982,226772,PotassiumApacheIIValue,PotassiumApacheIIValue,Scores - APACHE II,NaN,Numeric,17,"Max: 5.0, Min: 3.0, Mean: 3.9647058823529413"
1986,226776,SodiumApacheIIValue,SodiumApacheIIValue,Scores - APACHE II,NaN,Numeric,17,"Max: 150.0, Min: 136.0, Mean: 142.05882352941177"
2067,227024,MapScore_ApacheIV,MapScore_ApacheIV,Scores - APACHE IV (2),NaN,Numeric,8,"Max: 15.0, Min: 0.0, Mean: 7.75"
2066,227023,MAP_ApacheIV,MAP_ApacheIV,Scores - APACHE IV (2),mmHg,Numeric,7,"Max: 122.0, Min: 54.0, Mean: 73.85714285714286"


In [11]:
ItemFinder("Mean art").candidate_table

,itemid,label,abbreviation,category,unitname,param_type,count,value_instances


### sbp

In [6]:
ItemFinder("systolic").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
24,220179,Non Invasive Blood Pressure systolic,NBPs,Routine Vital Signs,mmHg,Numeric,4067219.0
6,220050,Arterial Blood Pressure systolic,ABPs,Routine Vital Signs,mmHg,Numeric,2264001.0
11,220059,Pulmonary Artery Pressure systolic,PAPs,Hemodynamics,mmHg,Numeric,309026.0
1212,225309,ART BP Systolic,ART BP Systolic,Routine Vital Signs,mmHg,Numeric,228488.0
2694,228152,Aortic Pressure Signal - Systolic,Aortic Pressure Signal - Systolic,Impella,mmHg.,Numeric,8684.0
575,224167,Manual Blood Pressure Systolic Left,Manual BPs L,Routine Vital Signs,mmHg,Numeric,1400.0
2155,227243,Manual Blood Pressure Systolic Right,Manual BPs R,Routine Vital Signs,mmHg,Numeric,1136.0
3778,229669,Pulmonary Atrtery Pressure Signal - Systolic,Pulmonary Artery Pressure Signal - Systolic,Impella,mmHg.,Numeric,848.0
1995,226850,RV systolic pressure(PA Line),RV systolic pressure(PA Line),PA Line Insertion,mmHg,Numeric,NaN
1997,226852,PA systolic pressure(PA Line),PA systolic pressure(PA Line),PA Line Insertion,mmHg,Numeric,NaN


### dbp

In [5]:
ItemFinder("diastolic").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
25,220180,Non Invasive Blood Pressure diastolic,NBPd,Routine Vital Signs,mmHg,Numeric,4066337.0
7,220051,Arterial Blood Pressure diastolic,ABPd,Routine Vital Signs,mmHg,Numeric,2263647.0
12,220060,Pulmonary Artery Pressure diastolic,PAPd,Hemodynamics,mmHg,Numeric,309126.0
1213,225310,ART BP Diastolic,ART BP Diastolic,Routine Vital Signs,mmHg,Numeric,228417.0
2693,228151,Aortic Pressure Signal - Diastolic,Aortic Pressure Signal - Diastolic,Impella,mmHg.,Numeric,8678.0
768,224643,Manual Blood Pressure Diastolic Left,Manual BPd L,Routine Vital Signs,mmHg,Numeric,1296.0
2154,227242,Manual Blood Pressure Diastolic Right,Manual BPd R,Routine Vital Signs,mmHg,Numeric,1042.0
3777,229668,Pulmonary Artery Pressure Signal - Diastolic,Pulmonary Artery Pressure Signal - Diastolic,Impella,mmHg.,Numeric,849.0
1996,226851,RV diastolic pressure(PA Line),RV diastolic pressure(PA Line),PA Line Insertion,mmHg,Numeric,NaN
1998,226853,PA diastolic pressure(PA Line),PA diastolic pressure(PA Line),PA Line Insertion,mmHg,Numeric,NaN


### `pulse`

In [8]:
ItemFinder('pulse', col="abbr").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
430,223934,Dorsal PedPulse R,Dorsal PedPulse R,Cardiovascular (Pulses),NaN,Text,1027492
438,223943,Dorsal PedPulse L,Dorsal PedPulse L,Cardiovascular (Pulses),NaN,Text,1013331
431,223935,PostTib. Pulses R,PostTib Pulses R,Cardiovascular (Pulses),NaN,Text,757423
442,223947,PostTib. Pulses L,PostTib Pulses L,Cardiovascular (Pulses),NaN,Text,744884
432,223936,Radial Pulse R,Radial Pulse R,Cardiovascular (Pulses),NaN,Text,514582
443,223948,Radial Pulse L,Radial Pulse L,Cardiovascular (Pulses),NaN,Text,505331
433,223938,Ulnar Pulse R,Ulnar Pulse R,Cardiovascular (Pulses),NaN,Text,20949
444,223949,Ulnar Pulse L,Ulnar Pulse L,Cardiovascular (Pulses),NaN,Text,18107
435,223940,Femoral Pulse R,Femoral Pulse R,Cardiovascular (Pulses),NaN,Text,13552
434,223939,Brachial Pulse R,Brachial Pulse R,Cardiovascular (Pulses),NaN,Text,12171


In [25]:
ItemFinder('heart rate').create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
2,220045,Heart Rate,HR,Routine Vital Signs,bpm,Numeric,6460839
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,Alarms,bpm,Numeric,627396
3,220046,Heart rate Alarm - High,HR Alarm - High,Alarms,bpm,Numeric,627111


In [76]:
find_value_instances(223934)

item label is Dorsal PedPulse R


Index(['Easily Palpable', 'Doppler', 'Weak Palpable', 'Strong/Palpable',
       'Difficult to Palpate', 'Absent', 'Not Applicable'],
      dtype='object', name='value')

In [37]:
ItemFinder.find_value_instances(229770)

item label is Resting Pulse Rate (COWS)


Index(['Pulse rate 80 or below', 'Pulse rate 81-100', 'Pulse rate 101-120',
       'Pulse rate greater than 120'],
      dtype='object', name='value')

### spo2

In [10]:
ItemFinder('spo2', col = "abbr").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
36,220277,O2 saturation pulseoxymetry,SpO2,Respiratory,%,Numeric,6324341
346,223770,O2 Saturation Pulseoxymetry Alarm - Low,SpO2 Alarm - Low,Alarms,%,Numeric,622764
345,223769,O2 Saturation Pulseoxymetry Alarm - High,SpO2 Alarm - High,Alarms,%,Numeric,619601
1810,226253,SpO2 Desat Limit,SpO2 Desat Limit,Alarms,%,Numeric,600821
3920,229862,Forehead SpO2 Sensor in Place,Forehead SpO2 Sensor in Place,Routine Vital Signs,NaN,Checkbox,35


### `height_inches`

In [12]:
ItemFinder("height").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
1941,226707,Height,Height,General,Inch,Numeric,33707
1945,226730,Height (cm),Height (cm),General,cm,Numeric,33707


## `labs`

In [13]:
ItemFinder("protein").create_candidate_table()

,itemid,label,abbreviation,category,unitname,param_type,count
2205,227444,C Reactive Protein (CRP),C Reactive Protein (CRP),Labs,NaN,Numeric with tag,2749.0
157,220650,Total Protein,Total Protein,Labs,NaN,Numeric,2478.0
1794,226184,Estimated Protein Needs/Kg,Estimated Protein Needs/Kg,General,g/kg,Numeric,450.0
93,220454,Protein,Protein,Ingredients,grams,Ingredient,NaN
148,220612,ZC Reactive Protein (CRP),ZC Reaction Protein (CRP),Labs,NaN,Numeric,NaN
1667,225970,Beneprotein,Beneprotein,Nutrition - Enteral,grams,Solution,NaN
3514,229296,Vital High Protein (Full),Vital High Protein (Full),Nutrition - Enteral,mL,Solution,NaN
3713,229583,Beneprotein.,Beneprotein.,Nutrition - Supplements,mL,Solution,NaN


## `respiratory_support`

In [11]:
resp_df = ItemFinder("Respiratory", col="category").candidate_table
resp_df

item label is Respiratory Rate
value instances are Max: 2355560.0, Min: 0.0, Mean: 20.459964592066456
item label is O2 saturation pulseoxymetry
value instances are Max: 9900000.0, Min: -272.0, Mean: 104.82763597187437
item label is O2 Delivery Device(s)
value instances are ['Endotracheal tube', 'Nasal cannula', 'Tracheostomy tube', 'Aerosol-cool', 'Face tent', 'Trach mask ', 'High flow nasal cannula', 'Bipap mask ', 'High flow neb', 'Non-rebreather', 'CPAP mask ', 'Venti mask ', 'Other', 'Medium conc mask ', 'Oxymizer', 'T-piece', 'Ultrasonic neb', 'Vapomist']
item label is Inspired O2 Fraction
value instances are Max: 10050.0, Min: 0.0, Mean: 48.94540639505812
item label is PEEP set
value instances are Max: 8774580.0, Min: -5.0, Mean: 26.214116360816025
item label is Tidal Volume (observed)
value instances are Max: 709461.0, Min: 0.0, Mean: 470.8951592275197
item label is Minute Volume
value instances are Max: 18918.4, Min: 0.0, Mean: 9.379572454568208
item label is Mean Airway Pressu

,itemid,label,abbreviation,linksto,category,unitname,param_type,count,value_instances
28,220210,Respiratory Rate,RR,chartevents,Respiratory,insp/min,Numeric,6393762.0,"Max: 2355560.0, Min: 0.0, Mean: 20.45996459206..."
36,220277,O2 saturation pulseoxymetry,SpO2,chartevents,Respiratory,%,Numeric,6324341.0,"Max: 9900000.0, Min: -272.0, Mean: 104.8276359..."
1946,226732,O2 Delivery Device(s),O2 Delivery Device(s),chartevents,Respiratory,NaN,Text,1536870.0,"['Endotracheal tube', 'Nasal cannula', 'Trache..."
384,223835,Inspired O2 Fraction,FiO2,chartevents,Respiratory,NaN,Numeric,849875.0,"Max: 10050.0, Min: 0.0, Mean: 48.94540639505812"
40,220339,PEEP set,PEEP set,chartevents,Respiratory,cmH2O,Numeric,642089.0,"Max: 8774580.0, Min: -5.0, Mean: 26.2141163608..."
...,...,...,...,...,...,...,...,...,...
3969,229992,Filters changed (Anes),Filters changed (Anes),chartevents,Respiratory,NaN,Checkbox,NaN,Checkbox
3970,229993,Vaporizer filled (Anes),Vaporizer filled (Anes),chartevents,Respiratory,NaN,Checkbox,NaN,Checkbox
3971,229994,Vaporizer checked (Anes),Vaporizer checked (Anes),chartevents,Respiratory,NaN,Checkbox,NaN,Checkbox
3972,229995,Amsorb changed (Anes),Amsorb changed (Anes),chartevents,Respiratory,NaN,Checkbox,NaN,Checkbox
